<a href="https://colab.research.google.com/github/panditrahulsharma/Blood-Cell-Subtypes-Classification-Using-deep-learning-and-CNN-model/blob/master/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.getcwd()
os.chdir('/content/drive/My Drive/blood_cell/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Pickle in Python is primarily used in serializing and deserializing a Python object structure. In other words, 
# it's the process of converting a Python object into a byte stream to store it in a file/database, maintain program\
# state across sessions, or transport data over the network.
#To open the file for writing, simply use the open() function. The first argument should be the name of your file. The 
# second argument is 'wb'. The w means that you'll be writing to the file, and b refers to binary mode.
#Once the file is opened for writing, you can use pickle.dump(), which takes two arguments: the object you want 
#to pickle and the file to which the object has to be saved. In this case, the former will be classifier, while 
# the latter will be model_pkl.
# Close the pickle instances using pickle.close()


import pickle
digit_detect_pkl=open('Blood_cell.pkl','rb') #opening our pickle file of model in "rb" i.e. "rb" mode opens the file in binary format for reading.
model = pickle.load(digit_detect_pkl) #loading our pickle object using .load method of pickle.

In [0]:
#import library
from keras.preprocessing import image
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import tensorflow.compat.v1 as tf
from keras.models import load_model


In [0]:
in_dir='/content/drive/My Drive/blood_cell/dataset2-master/dataset2-master/images/'
train_dataset_path=in_dir+"TRAIN/"
test_dataset_path=in_dir+"TEST/"
CATEGORIES    = ['EOSINOPHIL','LYMPHOCYTE','MONOCYTE','NEUTROPHIL']


In [69]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory(test_dataset_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            color_mode='rgb',
                                            class_mode = 'categorical')


Found 2487 images belonging to 4 classes.


In [0]:
y_pred = model.predict_generator(test_set) #making new prediction on our test_set using .predict_generator method from Keras ImageDataGenerator 
y_pred=y_pred.argmax(axis=-1) #argmax returns the index of maximum value whereas axix=-1 signifies that 
                              #that the index that will be returned by argmax will be taken from the last axis. 

In [0]:
y_true = test_set.classes #using .classes method of test_set to get all the actual classes indices for comparing 
                          #with predicted values.


In [72]:
y_true

array([0, 0, 0, ..., 3, 3, 3], dtype=int32)

# CLASSIFICATION METRICS :


# Classification Accuracy
Classification Accuracy is what we usually mean, when we use the term accuracy. It is the ratio of number of correct predictions to the total number of input samples.
![](https://miro.medium.com/max/373/1*yRa2inzTnyASJOre93ep3g.gif)

It works well only if there are equal number of samples belonging to each class.
For example, consider that there are 98% samples of class A and 2% samples of class B in our training set. Then our model can easily get 98% training accuracy by simply predicting every training sample belonging to class A.
When the same model is tested on a test set with 60% samples of class A and 40% samples of class B, then the test accuracy would drop down to 60%. Classification Accuracy is great, but gives us the false sense of achieving high accuracy.
The real problem arises, when the cost of misclassification of the minor class samples are very high. If we deal with a rare but fatal disease, the cost of failing to diagnose the disease of a sick person is much higher than the cost of sending a healthy person to more tests.

# Logarithmic Loss
Logarithmic Loss or Log Loss, works by penalising the false classifications. It works well for multi-class classification. When working with Log Loss, the classifier must assign probability to each class for all the samples. Suppose, there are N samples belonging to M classes, then the Log Loss is calculated as below :
![](https://miro.medium.com/max/344/1*dtpzlB_dNrmxDq7fGnJ4cg.gif)
where,
y_ij, indicates whether sample i belongs to class j or not
p_ij, indicates the probability of sample i belonging to class j
Log Loss has no upper bound and it exists on the range [0, ∞). Log Loss nearer to 0 indicates higher accuracy, whereas if the Log Loss is away from 0 then it indicates lower accuracy.
In general, minimising Log Loss gives greater accuracy for the classifier.

# Confusion Matrix
Confusion Matrix as the name suggests gives us a matrix as output and describes the complete performance of the model.
Lets assume we have a binary classification problem. We have some samples belonging to two classes : YES or NO. Also, we have our own classifier which predicts a class for a given input sample. On testing our model on 165 samples ,we get the following result.
![](https://miro.medium.com/max/386/1*GMlSubndVt3g7FmeQjpeMA.png)

There are 4 important terms :
True Positives : The cases in which we predicted YES and the actual output was also YES.
True Negatives : The cases in which we predicted NO and the actual output was NO.
False Positives : The cases in which we predicted YES and the actual output was NO.
False Negatives : The cases in which we predicted NO and the actual output was YES.
Accuracy for the matrix can be calculated by taking average of the values lying across the “main diagonal” i.e.
![](https://miro.medium.com/max/366/1*NDD6vdbTcLPzXVbCKi87RA.gif)
![](https://miro.medium.com/max/241/1*VbjXSHvKxtgoi-oM2OHuuQ.gif)

Confusion Matrix forms the basis for the other types of metrics.

# Area Under Curve
Area Under Curve(AUC) is one of the most widely used metrics for evaluation. It is used for binary classification problem. AUC of a classifier is equal to the probability that the classifier will rank a randomly chosen positive example higher than a randomly chosen negative example. Before defining AUC, let us understand two basic terms :

**True Positive Rate (Sensitivity)** : True Positive Rate is defined as TP/ (FN+TP). True Positive Rate corresponds to the proportion of positive data points that are correctly considered as positive, with respect to all positive data points.
![](https://miro.medium.com/max/420/1*yw4Y3D7nGNVza2EC2WrOfg.gif)

**False Positive Rate (Specificity)** : False Positive Rate is defined as FP / (FP+TN). False Positive Rate corresponds to the proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points.
![](https://miro.medium.com/max/426/1*eIIZS8FNvpxiEzHpFI3QZA.gif)

False Positive Rate and True Positive Rate both have values in the range [0, 1]. FPR and TPR bot hare computed at threshold values such as (0.00, 0.02, 0.04, …., 1.00) and a graph is drawn. AUC is the area under the curve of plot False Positive Rate vs True Positive Rate at different points in [0, 1].
![](https://miro.medium.com/max/640/1*zFW1Kj3e2X_mmluTW3rVeA.png)
As evident, AUC has a range of [0, 1]. The greater the value, the better is the performance of our model.

# F1 Score
F1 Score is used to measure a test’s accuracy.F1 Score is the Harmonic Mean between precision and recall. The range for F1 Score is [0, 1]. It tells you how precise your classifier is (how many instances it classifies correctly), as well as how robust it is (it does not miss a significant number of instances).
High precision but lower recall, gives you an extremely accurate, but it then misses a large number of instances that are difficult to classify. The greater the F1 Score, the better is the performance of our model. Mathematically, it can be expressed as :
![](https://miro.medium.com/max/191/1*_pYttqYh8w-EpLxMi84H8A.gif)

F1 Score tries to find the balance between precision and recall.

**Precision** : It is the number of correct positive results divided by the number of positive results predicted by the classifier.
![](https://miro.medium.com/max/365/1*wMIDfGwT9bA6HezvYhdbpQ.gif)


**Recall** : It is the number of correct positive results divided by the number of all relevant samples (all samples that should have been identified as positive).
![](https://miro.medium.com/max/371/1*gIlQMZBPjtUHWkwOhRG3KA.gif)

# Mean Absolute Error
Mean Absolute Error is the average of the difference between the Original Values and the Predicted Values. It gives us the measure of how far the predictions were from the actual output. However, they don’t gives us any idea of the direction of the error i.e. whether we are under predicting the data or over predicting the data. Mathematically, it is represented as : ![](https://miro.medium.com/max/303/1*qak4Dadzs7pO0hnz4q8O8Q.gif)

# Mean Squared Error
Mean Squared Error(MSE) is quite similar to Mean Absolute Error, the only difference being that MSE takes the average of the square of the difference between the original values and the predicted values. The advantage of MSE being that it is easier to compute the gradient, whereas Mean Absolute Error requires complicated linear programming tools to compute the gradient. As, we take square of the error, the effect of larger errors become more pronounced then smaller error, hence the model can now focus more on the larger errors.
![](https://miro.medium.com/max/312/1*okvAVQNY6s5cMHxrqUzM5A.gif)





In [0]:
from sklearn.metrics import confusion_matrix,accuracy_score #importing confusion_matrix and accuracy_score from sklearn.metrics


In [83]:
cm=confusion_matrix(y_true,y_pred) #confusion matrix takes 2 arguments i.e. actual and predicted values
print(cm)

[[ 41 121 355 106]
 [ 53 110 342 115]
 [ 48 127 336 109]
 [ 50 111 370  93]]


In [86]:
print(accuracy_score(y_true,y_pred)) #accuracy_score takes 2 arguments i.e. actual and predicted values

0.23321270607157218


# SUMMARY 

Our aim was to develop a deep learning model to identify four different types of White Blood cells found in humans that are Eosinophils , Monocytes , Lymphocytes and Neutrophils. Primary approach towards this problem was to develop a Convolutional Neural Network model that can classify blood smear slides into four categories mentioned above.Deep learning framework which we used for this problem was Keras.We used ImageDataGenerator class of Keras to load images directly from directory and for Image Preprocessing purposes like Rescaling.OpenCV was used to display the images and have a better intuition about the problem.A 14 layer deep CNN was implemented in which we used Dropout layers for regularization purpose to prevent overfitting.as being a multiclass classification problem we used softmax activation function in the last Dense layer followed by categorical_crossentropy as loss function and accuracy as classification metrics.After training of CNN model we managed to get a very good training and validation accuracy of 92.66% and 90.19% respectively.

